# Summary
This notebook applies SS-CASTLE, mutual information, partial correlation, Pearson's correlation, DTF, and PDC to the data generated in `2.1_generate_synth_data.ipynb`.

In [1]:
import sys
sys.path.insert(0, r"..\code")

import numpy as np

from utils import save_obj, load_obj

from MSCASTLE import SSCASTLE
from metrics import get_DAG, count_accuracy
from pdc_dtf import PDC, DTF, mvar_fit
from tqdm.notebook import tqdm
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt

In [2]:
data_dir="../data/"
ts_dir="../data/TimeSeriesAAL/" 
processed="../data/processed/"
diffreg="../data/processed/diff_regions/"

In [3]:
data=load_obj("2.1_data", processed)

print(data['info'])

S,K,T=data[0]['X_skt'].shape
nsims=50

S: # of subjects;
K: # of ts;
T: length of ts;
X_skt: data generated from the causal structure;
C_skk: causal structure;
B_kk: backbone.


In [4]:
replace=False
verbose=False

# Simulate

In [5]:
lmbds=[1.e-3, 5.e-3, 1.e-2, 5.e-2, 1.e-1, 5.e-1]

if replace:
    
    results=dict()
    
    for sim in tqdm(range(nsims)):
        X_skt=data[sim]['X_skt']
        C_skk=data[sim]['C_skk']

        results[sim]=dict()

        for s in tqdm(range(S)):
            
            results[sim][s]=dict()
            ts = X_skt[s].T

            for lmbd in lmbds:
                results[sim][s][lmbd]=dict()
                sscastle = SSCASTLE(ts, lag=0)
                C,_ = sscastle.solver(reg='l1', lmbd=lmbd, thresh=.1, h_tol=1.e-10, interval=(-1., 1.), verbose=verbose)
                results[sim][s][lmbd]['C_hat']=np.squeeze(C)
                results[sim][s][lmbd]['metrics causal discovery'] = count_accuracy(get_DAG(C_skk[s]), get_DAG(results[sim][s][lmbd]['C_hat']))    
                
            mutual_info_matrix = np.eye(K)
            for i in range(K-1):
                mutual_info_matrix[i,i+1:] += mutual_info_regression(ts[...,i+1:], ts[...,i])
            mutual_info_matrix+=np.triu(mutual_info_matrix,k=1).T
            results[sim][s]['MI']=mutual_info_matrix
            
            P = np.linalg.inv(np.cov(ts, rowvar=False))
            D = np.diag(np.diag(P)**(-1/2))
            R = - D @ P @ D
            results[sim][s]['PC'] = R

            results[sim][s]['Corr'] = np.corrcoef(ts, rowvar=False) 

            A_est, sigma = mvar_fit(ts.T, p=1)
            sigma = np.diag(sigma)  # DTF + PDC support diagonal noise

            #DTF
            dtf, _ = DTF(A_est, sigma)
            dtf1=np.linalg.norm(dtf, ord=2, axis=0)
            ddtf1=np.diag(np.diag(dtf1)**(-.5))
            dtf2=ddtf1@dtf1@ddtf1
            results[sim][s]['DTF'] = dtf2.T
            
            #PDC
            pdc, _ = PDC(A_est, sigma)
            pdc1=np.linalg.norm(pdc, ord=2, axis=0)
            dpdc1=np.diag(np.diag(pdc1)**(-.5))
            pdc2=dpdc1@pdc1@dpdc1
            results[sim][s]['PDC'] = pdc2.T

    save_obj(results, "2.2_synth_results", processed)
else:
    results=load_obj("2.2_synth_results", processed)